In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from ipywidgets import interact, FloatSlider, IntSlider
from types import SimpleNamespace
import sympy as sp
from IPython.display import display
from OpenEconomy1 import OpenEconomy

# Model Project - The Small Open Economy

Set-up is as follows: GDP, the national identity, is defined as the sum of consumption, investment and net-exports:
$$Y = C + I + NX$$   
GNI adds to this rental income from foreign capital:
$$Y + rF =  C + I + NX + rF$$    
Savings are Gross National Income minus consumption: 
$$S_t = Y_t + rF_t - C_t$$
Savings help accumulate capital - either domestically, or abroad: 
$$S_t = I_t + F_{t+1} - F_t$$
Capital evolves as per usual (at first simplified by assuming zero depreciation ... assumption later relaxed):
$$K_{t+1} = I_t + K_t$$
Combining terms we see that:
$$K_{t+1} = S_t - F_{t+1} + F_t + K_t ↔ K_{t+1} + F_{t+1} = S_t + F_t + K_t$$
Wealth can be domestically owned, or foreign:
$$V_t = K_t + F_t$$
This means that wealth tomorrow is:
$$V_{t+1} = V_t + S_t$$
$S_t = s (Y_t + rF_t)$, where saving, s, is a fraction of income set between 0 and 1.  Our production function is initially defined as CD (we'll show other variants later) $Y_t = AK_t^{\alpha} L_t^{1-\alpha}$. Markets are competitive. $r=r^w$ at all times. This implies a constant level of capital (barring changes to $\alpha$ or A). 
$MP_K$ = $f'k$ = rental rate of capital. Thus, $f'k = \alpha A K_t^{\alpha -1} L_t^{1-\alpha}$, $r^w = \alpha A k_t^{\alpha - 1}$, and $\bar{k} = \left(\frac{r^w}{\alpha A}\right)^{\frac{1}{\alpha-1}}$. Wages are constant by the same reasoning. $MP_L$ = $f'l$ = $\bar{w}$ = $(1-{\alpha})$ $\frac{Y}{L}$ = $(1-{\alpha})A \bar{k}^{\alpha}$. Total returns to scale tell us that $Y_t = rK_t + wL_t$. 

We use sympy to derive the law of motion for wealth and then proceed to the numerical analysis the assignment asks for. 

Equations come from: https://web.econ.ku.dk/dalgaard/makro2_2008/Slides/lecture4_cm.pdf

In [2]:
# Defining the necessary symbols
Y_t, C_t, I_t, r, rw, F_t, K_t, S, s, V_t, V1, L_t, w, n, L_t1, v_t = sp.symbols(
    'Y_t C_t I_t r r^w F_t K_t S s V_t V_{t+1} L_t w n L_{t+1} v_t'
)

# Equation setup
Y = w * L_t + r * K_t         # Production function
GNI = Y + r * F_t               # Gross National Income
S = s * GNI                     # Savings
V = K_t + F_t                   # Wealth at time t
V1 = S + V                      # Wealth at time t+1

# Expand all expressions 
V1_expanded = V1.expand()

# Substitute V_t for K_t + F_t in the expanded V1
V1_substituted = V1_expanded.subs({K_t + F_t: V_t})

# Collect terms to consolidate expressions around V_t
V1_collected = sp.collect(V1_substituted, V_t)

# Explicitly factor out (1 + r * s) from the terms involving V_t
V1_intermed = V1_collected.subs(r * s * K_t + r * s * F_t, (r * s + 1) * V_t).simplify()

# Define L_t+1 
L_1 = L_t * (1 + n)

# Define per capita wealth today and tomorrow 
v_t = V_t / L_t
v_1 = V1_intermed / L_1

# This gives us the Law of Motion for Wealth (LoM)
LoM = v_1.subs(r * s * K_t + r * s * F_t, (r * s + 1) * V_t).simplify()

LoM

(L_t*s*w + V_t*(r*s + 1) + V_t)/(L_t*(n + 1))

We divide this thru by $L_t$ to arrive at: $v_{t+1} = \frac{sw}{1+n}+\frac{1+sr}{1+n}v_t$

In the Steady State, $v_{t+1}=v_t=v^*$

In [3]:
s, w, r, n, v = sp.symbols('s w r n v')
SteadyState = sp.Eq((s * w) / (1 + n) + ((1 + s * r) / (1 + n)) * v, v)
vstar = sp.solve(SteadyState, v)
vstar

[s*w/(n - r*s)]

## Numerical Analysis 




### Phase Diagram

**Use sliders to see reaction to changes in parameters**

In [4]:
model = OpenEconomy()

interact(model.phase_diagram,
         s=FloatSlider(min=0, max=1, step=0.01, value=model.s, description='Savings Rate (s)'),
         n=FloatSlider(min=0, max=0.1, step=0.001, value=model.n, description='Population Growth Rate (n)'),
         alpha=FloatSlider(min=0, max=1, step=0.01, value=model.alpha, description='Output Elasticity of Capital (alpha)'),
         A=FloatSlider(min=0.5, max=5, step=0.1, value=model.A, description='Technology Parameter (A)'),
         delta=FloatSlider(min=0, max=0.1, step=0.01, value=model.delta, description='Depreciation Rate (delta)'))


interactive(children=(FloatSlider(value=0.2, description='Savings Rate (s)', max=1.0, step=0.01), FloatSlider(…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

# Underlying variables

**Change parameters to see reaction and change "shock" in order to determine which period**

In [5]:
interact(model.underlying_variables,
         periods=IntSlider(min=50, max=500, step=50, value=100, description='Periods'),
         shock=IntSlider(min=0, max=99, step=1, value=None, description='Shock Time', style={'description_width': 'initial'}),
         s=FloatSlider(min=0.0, max=1.0, step=0.01, value=model.s, description='Savings Rate (s)'),
         n=FloatSlider(min=0.0, max=0.1, step=0.001, value=model.n, description='Population Growth Rate (n)'),
         alpha=FloatSlider(min=0.0, max=1.0, step=0.01, value=model.alpha, description='Output Elasticity of Capital (alpha)'),
         A=FloatSlider(min=0.5, max=5.0, step=0.1, value=model.A, description='Technology Parameter (A)'),
         delta=FloatSlider(min=0.0, max=0.1, step=0.01, value=model.delta, description='Depreciation Rate (delta)'))

interactive(children=(IntSlider(value=100, description='Periods', max=500, min=50, step=50), IntSlider(value=0…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

## Convergence to Steady State

In [6]:
model.reset() 


interact(model.convergence_ss,
         periods=IntSlider(min=50, max=500, step=50, value=100, description='Periods'),
         shock=IntSlider(min=0, max=100, step=1, value=None, description='Shock Time', style={'description_width': 'initial'}),
         s=FloatSlider(min=0, max=1, step=0.01, value=model.s, description='Savings Rate (s)'),
         n=FloatSlider(min=0, max=0.1, step=0.001, value=model.n, description='Population Growth Rate (n)'),
         alpha=FloatSlider(min=0, max=1, step=0.01, value=model.alpha, description='Output Elasticity of Capital (alpha)'),
         A=FloatSlider(min=0.5, max=5, step=0.1, value=model.A, description='Technology Parameter (A)'),
         delta=FloatSlider(min=0, max=0.1, step=0.01, value=model.delta, description='Depreciation Rate (delta)'))

interactive(children=(IntSlider(value=100, description='Periods', max=500, min=50, step=50), IntSlider(value=0…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

# Introducing shocks from Netto Export

The shock from nettoexport in this model is determind by competitiveness of exhange rates which is affected by the differences in interest rates and inflation rates

In [7]:
from OpenEconomy2 import OpenEconomyNX
from ipywidgets import interact, IntSlider, FloatSlider

model = OpenEconomyNX()


interact(model.nx_phase_diagram,
         alpha=FloatSlider(min=0.1, max=0.9, step=0.05, value=model.alpha, description='Alpha (Output Elasticity)'),
         n=FloatSlider(min=0.01, max=0.05, step=0.005, value=model.n, description='n (Population Growth Rate)'),
         s=FloatSlider(min=0.1, max=0.4, step=0.05, value=model.s, description='s (Savings Rate)'),
         r=FloatSlider(min=0.01, max=0.05, step=0.005, value=model.r, description='r (Interest Rate)'),
         foreign_inflation_rate=FloatSlider(min=0.0, max=0.05, step=0.005, value=model.foreign_inflation_rate, description='Foreign Inflation Rate'),
         domestic_inflation_rate=FloatSlider(min=0.0, max=0.05, step=0.005, value=model.domestic_inflation_rate, description='Domestic Inflation Rate'),
         delta=FloatSlider(min=0.01, max=0.1, step=0.01, value=model.delta, description='Delta (Depreciation Rate)'))


interactive(children=(FloatSlider(value=0.3, description='Alpha (Output Elasticity)', max=0.9, min=0.1, step=0…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [8]:
from OpenEconomy2 import OpenEconomyNX
from ipywidgets import interact, IntSlider, FloatSlider

model = OpenEconomyNX()


interact(model.nx_underlying_variables,
         periods=IntSlider(min=50, max=500, step=10, value=100, description='Periods'),
         shock=IntSlider(min=0, max=100, step=1, value=0, description='Shock Time'),
         alpha=FloatSlider(min=0.1, max=0.9, step=0.01, value=model.alpha, description='Alpha'),
         n=FloatSlider(min=0, max=0.1, step=0.01, value=model.n, description='Population Growth (n)'),
         s=FloatSlider(min=0.1, max=0.4, step=0.01, value=model.s, description='Savings Rate (s)'),
         r=FloatSlider(min=0.01, max=0.1, step=0.01, value=model.r, description='Interest Rate (r)'),
         A=FloatSlider(min=0.5, max=2.0, step=0.1, value=model.A, description='Technology Factor (A)'),
         delta=FloatSlider(min=0.01, max=0.1, step=0.01, value=model.delta, description='Depreciation Rate (delta)'),
         foreign_inflation_rate=FloatSlider(min=0, max=0.1, step=0.01, value=model.foreign_inflation_rate, description='Foreign Inflation Rate'),
         domestic_inflation_rate=FloatSlider(min=0, max=0.1, step=0.01, value=model.domestic_inflation_rate, description='Domestic Inflation Rate'))

interactive(children=(IntSlider(value=100, description='Periods', max=500, min=50, step=10), IntSlider(value=0…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>